In [1]:
# SimpleWriteCode 这个类与上一节一模一样

from metagpt.actions import Action

class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

2025-01-23 19:28:01.173 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to e:\wow-agent\notebook


In [2]:
# 本节新增了SimpleRunCode这个类
class SimpleRunCode(Action):
    name: str = "SimpleRunCode"

    async def run(self, code_text: str):
        result = subprocess.run(["python", "-c", code_text], capture_output=True, text=True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

In [3]:
import re
import os
import subprocess
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger
class RunnableCoder(Role):
    name: str = "Alice"
    profile: str = "RunnableCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteCode, SimpleRunCode])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0]  # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=result, role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg

In [4]:
async def main():
    msg = "write a function that calculates the sum of a list"
    role = RunnableCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)
    return result

rtn = await main()
print(rtn)

2025-01-23 19:28:31.277 | INFO     | __main__:main:4 - write a function that calculates the sum of a list
2025-01-23 19:28:31.279 | INFO     | __main__:_act:17 - Alice(RunnableCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def sum_list(numbers):
    return sum(numbers)

# Test case 1
assert sum_list([1, 2, 3, 4, 5]) == 15, "Test case 1 failed"

# Test case 2
assert sum_list([-1, -2, -3, -4, -5]) == -15, "Test case 2 failed"

print("All test cases passed!")
```


2025-01-23 19:28:33.015 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-23 19:28:33.017 | INFO     | __main__:_act:17 - Alice(RunnableCoder): to do SimpleRunCode(SimpleRunCode)
2025-01-23 19:28:33.075 | INFO     | __main__:run:8 - code_result='All test cases passed!\n'
2025-01-23 19:28:33.076 | INFO     | __main__:main:6 - RunnableCoder: All test cases passed!



RunnableCoder: All test cases passed!

